In [ ]:
import os
import sys
import nibabel as nib
from nilearn.image import concat_imgs, smooth_img
from nipype.interfaces.fsl import BET, MCFLIRT, SliceTimer

sys.path.append(os.path.join(os.getcwd(), '..', 'src', 'preprocess'))
from coregistrate import coregister_fmri_to_mni


def preprocess_data():
   # Define paths relative to the src folder
    base_dir = os.path.join("..", "data", "raw", "sub-control01")  # Navigate up one directory and then into data
    preprocess_dir = os.path.join("..", "data", "preprocessed", "sub-control01")  # Preprocess folder
    preprocess_anat_dir = os.path.join(preprocess_dir, "anat")  # Anatomical folder
    preprocess_func_dir = os.path.join(preprocess_dir, "func")  # Functional folder
    anat_path = os.path.join(base_dir, "anat", "sub-control01_T1w.nii.gz")
    func_paths = [
        os.path.join(base_dir, "func", "sub-control01_task-music_run-1_bold.nii.gz"),
        os.path.join(base_dir, "func", "sub-control01_task-music_run-2_bold.nii.gz"),
        os.path.join(base_dir, "func", "sub-control01_task-music_run-3_bold.nii.gz")
    ]
    # Create preprocess directory if it doesn't exist
    if not os.path.exists(preprocess_dir):
        os.makedirs(preprocess_dir)
    if not os.path.exists(preprocess_anat_dir):
        os.makedirs(preprocess_anat_dir)
    if not os.path.exists(preprocess_func_dir):
        os.makedirs(preprocess_func_dir)

    # Perform skull stripping
    bet = BET(in_file=anat_path, out_file=os.path.join(preprocess_anat_dir,"brain_anat.nii.gz"), mask=True)
    bet.run()

    # Standardize and save each functional run
    standardized_func_paths = []
    for func_path in func_paths:
        img = nib.load(func_path)
        img_standard = (img.get_fdata() - img.get_fdata().mean()) / img.get_fdata().std()
        img_standard_nii = nib.Nifti1Image(img_standard, img.affine)
        standardized_path = os.path.join(preprocess_func_dir, os.path.basename(func_path))
        img_standard_nii.to_filename(standardized_path)
        standardized_func_paths.append(standardized_path)

    # Slice Timing Correction
    slice_timing_corrected_paths = []
    for func_path in standardized_func_paths:
        slicetimer = SliceTimer(
            in_file=func_path,
            out_file=os.path.join(preprocess_func_dir, "slicetimed_" + os.path.basename(func_path)),
            time_repetition=3,
            slice_direction=1,  # 1 for ascending, 2 for descending
            interleaved=True
        )
        slicetimer.run()
        slice_timing_corrected_paths.append(slicetimer.inputs.out_file)

    # Load and concatenate slice-timed functional images
    func_imgs = [nib.load(func_path) for func_path in slice_timing_corrected_paths]
    concat_img = concat_imgs(func_imgs)
    concat_img_path = os.path.join(preprocess_func_dir, "concatenated_func.nii.gz")
    concat_img.to_filename(concat_img_path)

    # Motion Correction
    mcflirt = MCFLIRT(in_file=concat_img_path, out_file=os.path.join(preprocess_func_dir, "motion_corrected.nii.gz"))
    mcflirt.run()

    # Smoothing
    smoothed_img = smooth_img(os.path.join(preprocess_func_dir, "motion_corrected.nii.gz"), fwhm=5)
    preproccessed_img_path = os.path.join(preprocess_func_dir, "preprocessed_data.nii.gz")
    smoothed_img.to_filename(preproccessed_img_path)

    # Coregistration to MNI space
    coregistered_img = coregister_fmri_to_mni(preproccessed_img_path, output_dir=preprocess_func_dir)

if __name__ == "__main__":
    preprocess_data()

241103-16:25:37,410 nipype.interface INFO:
	 stderr 2024-11-03T16:25:37.410036:Warning: An input intended to be a single 3D volume has multiple timepoints. Input will be truncated to first volume, but this functionality is deprecated and will be removed in a future release.
241103-16:25:40,857 nipype.interface INFO:
	 stderr 2024-11-03T16:25:40.857005:Warning: An input intended to be a single 3D volume has multiple timepoints. Input will be truncated to first volume, but this functionality is deprecated and will be removed in a future release.
